In [4]:
import pandas as pd

In [20]:
df = pd.read_csv('./data/Train.csv')
dftest = pd.read_csv('./data/Test.csv')

In [21]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [9]:
df.shape

(8068, 11)

In [11]:
dftest.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,B
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6,A
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6,A
3,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6,B
4,459001,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6,A


In [12]:
print(df.groupby(['Segmentation']).size().reset_index(name='Count'))

  Segmentation  Count
0            A   1972
1            B   1858
2            C   1970
3            D   2268


In [13]:
dftest.shape

(2627, 11)

In [14]:
print(dftest.groupby(['Segmentation']).size().reset_index(name='Count'))

  Segmentation  Count
0            A    846
1            B    550
2            C    472
3            D    759


In [15]:
print(df.groupby(['Var_1']).size().reset_index(name='Count'))

   Var_1  Count
0  Cat_1    133
1  Cat_2    422
2  Cat_3    822
3  Cat_4   1089
4  Cat_5     85
5  Cat_6   5238
6  Cat_7    203


In [1]:
import mlflow 

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("customer-segmentation")

2023/08/01 11:43:28 INFO mlflow.tracking.fluent: Experiment with name 'customer-segmentation' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/rashmi/Documents/rashmi/github_repos/AtoZ_ml_ops_course/mlops_project/mlops-product-reviews/01-model-dev-tracking-registry/mlflow/mlruns/1', creation_time=1690883008982, experiment_id='1', last_update_time=1690883008982, lifecycle_stage='active', name='customer-segmentation', tags={}>

In [16]:
df.dtypes

ID                   int64
Gender              object
Ever_Married        object
Age                  int64
Graduated           object
Profession          object
Work_Experience    float64
Spending_Score      object
Family_Size        float64
Var_1               object
Segmentation        object
dtype: object

In [22]:
df['segment_int'] = pd.Categorical(df['Segmentation']).codes

In [33]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,segment_int
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D,3
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A,0
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B,1
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B,1
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A,0


In [34]:
df.shape

(8068, 12)

In [35]:
df.dropna(axis = 0, inplace = True)

In [36]:
df.shape

(6665, 12)

In [37]:
df.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
segment_int        0
dtype: int64

In [87]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, test_size=0.2, random_state=0)

In [39]:
df_val.shape

(1333, 12)

In [40]:
print(df_val.groupby(['segment_int']).size().reset_index(name='Count'))

   segment_int  Count
0            0    320
1            1    302
2            2    369
3            3    342


In [88]:
y_train = df_train.segment_int.values
y_val = df_val.segment_int.values

In [42]:
y_train.shape

(5332,)

In [43]:
del df_train['segment_int']
del df_val['segment_int']

In [50]:
global_mean = df.segment_int.mean()

In [46]:
categorical = ['Gender', 'Ever_Married','Graduated', 'Profession', 'Spending_Score', 'Var_1']
numerical = ['Age','Work_Experience','Family_Size']

In [47]:
df[categorical].nunique()

Gender            2
Ever_Married      2
Graduated         2
Profession        9
Spending_Score    3
Var_1             7
dtype: int64

In [49]:
female_mean = df[df.Gender == 'Female'].segment_int.mean()
print('gender == female:', round(female_mean, 3))

male_mean = df[df.Gender == 'Male'].segment_int.mean()
print('gender == male:  ', round(male_mean, 3))

gender == female: 1.501
gender == male:   1.577


In [51]:
female_mean/global_mean

0.9728864890662524

In [52]:
male_mean/global_mean

1.0220329536769208

In [54]:
for col in categorical:
    df_group = df.groupby(by=col).segment_int.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)

,mean,diff,risk
Gender,,,
Female,1.501004,-0.041832,0.972886
Male,1.576829,0.033993,1.022033


,mean,diff,risk
Ever_Married,,,
No,1.822492,0.279656,1.181261
Yes,1.349899,-0.192937,0.874946


,mean,diff,risk
Graduated,,,
No,1.799255,0.256419,1.166200
Yes,1.397035,-0.145801,0.905498


,mean,diff,risk
Profession,,,
Artist,1.322080,-0.220755,0.856916
Doctor,1.511824,-0.031011,0.979900
Engineer,1.204467,-0.338368,0.780684
Entertainment,1.186650,-0.356186,0.769136
Executive,1.502970,-0.039865,0.974161
Healthcare,2.481894,0.939058,1.608657
Homemaker,1.565714,0.022879,1.014829
Lawyer,1.278000,-0.264836,0.828345
Marketing,2.077253,0.534418,1.346387


,mean,diff,risk
Spending_Score,,,
Average,1.433815,-0.109021,0.929337
High,1.364542,-0.178294,0.884438
Low,1.632908,0.090073,1.058381


,mean,diff,risk
Var_1,,,
Cat_1,1.682692,0.139857,1.090649
Cat_2,1.682320,0.139485,1.090408
Cat_3,1.512618,-0.030217,0.980414
Cat_4,1.537102,-0.005733,0.996284
Cat_5,1.581081,0.038245,1.024789
Cat_6,1.532618,-0.010217,0.993378
Cat_7,1.554217,0.011381,1.007377


In [55]:
from sklearn.feature_extraction import DictVectorizer

In [56]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [57]:
train_dict[0]

{'Gender': 'Female',
 'Ever_Married': 'No',
 'Graduated': 'Yes',
 'Profession': 'Entertainment',
 'Spending_Score': 'Low',
 'Var_1': 'Cat_6',
 'Age': 25,
 'Work_Experience': 9.0,
 'Family_Size': 2.0}

In [58]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [59]:
X_train = dv.transform(train_dict)

In [60]:
X_train.shape

(5332, 28)

In [61]:
dv.get_feature_names()

AttributeError: 'DictVectorizer' object has no attribute 'get_feature_names'

## Train Logistic Regression

In [62]:
from sklearn.linear_model import LogisticRegression

In [63]:
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [64]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [65]:
model.predict_proba(X_val)

array([[0.17030187, 0.04721118, 0.04187504, 0.74061191],
       [0.06240785, 0.06162109, 0.12716356, 0.7488075 ],
       [0.32599936, 0.29404296, 0.31889981, 0.06105787],
       ...,
       [0.30205535, 0.44056486, 0.16416089, 0.09321891],
       [0.34384022, 0.26383094, 0.29103423, 0.10129461],
       [0.27896936, 0.11827341, 0.07137877, 0.53137847]])

In [69]:
y_pred_lr = model.predict(X_val)

In [71]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [72]:
print("Accuracy Score: ", accuracy_score(y_pred_lr, y_val))
print(classification_report(y_pred_lr, y_val))

Accuracy Score:  0.5251312828207052
              precision    recall  f1-score   support

           0       0.48      0.44      0.46       353
           1       0.16      0.34      0.21       138
           2       0.71      0.56      0.63       466
           3       0.69      0.63      0.66       376

    accuracy                           0.53      1333
   macro avg       0.51      0.49      0.49      1333
weighted avg       0.59      0.53      0.55      1333



In [73]:
import pickle

In [74]:
with open('models/log_reg.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out)

In [75]:
with mlflow.start_run():
    mlflow.set_tag("developer", "rashmi")
    mlflow.log_param("train-data-path","./mlflow/data/Train.csv")

    model = LogisticRegression(solver='liblinear', random_state=1)
    model.fit(X_train, y_train)
    y_pred_lr = model.predict(X_val)

    accuracy_score = accuracy_score(y_pred_lr, y_val)
    mlflow.log_metric("accuracy", accuracy_score)

In [84]:
from sklearn.tree import DecisionTreeClassifier


In [99]:
X_train_nodv = df_train.drop(columns=['Segmentation', 'segment_int', 'ID','Gender', 'Ever_Married','Graduated', 'Profession', 'Spending_Score', 'Var_1'], axis=1)
X_val_nodv = df_val.drop(columns=['Segmentation', 'segment_int', 'ID','Gender', 'Ever_Married','Graduated', 'Profession', 'Spending_Score', 'Var_1'], axis=1)

In [100]:
X_train_nodv

,Age,Work_Experience,Family_Size
6675,25,9.0,2.0
1947,63,0.0,7.0
6205,26,0.0,2.0
5269,74,0.0,2.0
1400,27,10.0,3.0
...,...,...,...
5962,55,3.0,1.0
3947,58,1.0,2.0
1991,31,0.0,4.0
3158,35,8.0,2.0


In [114]:
type(y_pred_dt), type(y_val)

(numpy.ndarray, numpy.ndarray)

In [115]:
len(y_pred_dt), len(y_val)

(1333, 1333)

In [116]:
print(y_pred_dt), print(y_val)

[3 3 0 ... 2 0 3]
[3 1 1 ... 1 1 3]


(None, None)

In [117]:
with mlflow.start_run():
    mlflow.set_tag("developer", "rashmi")
    mlflow.log_param("train-data-path","./mlflow/data/Train.csv")

    model_ = DecisionTreeClassifier()
    model.fit(X_train_nodv, y_train)
    y_pred_dt = model.predict(X_val_nodv)

    accuracy_score = accuracy_score(y_pred_dt, y_val)
    mlflow.log_metric("accuracy", accuracy_score)

ValueError: Found input variables with inconsistent numbers of samples: [5332, 119]

In [104]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [105]:
model_dtclassifier = DecisionTreeClassifier()
model_dtclassifier.fit(X_train, y_train)
y_pred_dtclassifier = model_dtclassifier.predict(X_test)
 
print("Accuracy Score:", accuracy_score(y_pred_dtclassifier, y_test))
print(classification_report(y_pred_dtclassifier, y_test))

TypeError: 'numpy.float64' object is not callable

In [109]:
type(y_test)

pandas.core.series.Series

In [110]:
type(y_pred_dtclassifier)

numpy.ndarray